In [1]:
1+1

2

In [2]:
import mozo2 

import importlib
importlib.reload(mozo2)

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from mozo import compare_and_combine_2, Diff, merge_with_features, num_to_range_categories
import pandas as pd
import numpy as np
np.random.seed(0)

import xgboost as xgb

from scipy import stats
import math 
import datetime 

from sklearn.metrics import r2_score, mean_absolute_error
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from functools import partial
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from sklearn.svm import SVC

from sklearn.metrics import r2_score, f1_score, mean_absolute_error, mean_squared_error

from scikitplot.estimators import plot_learning_curve

import time
from datetime import datetime


In [3]:
## modelowanie

In [4]:
# import warnings
# warnings.filterwarnings("ignore")

total = pd.read_csv('~/pml7/konkurs/output/newest_total_20201101_07_20.h5.csv')
total.info(verbose=True)

train_30 = pd.read_hdf('output/train_30.h5', 'data') 
test_car_ids = train_30['car_id'].values

train_70 = total[~total['car_id'].isin(test_car_ids) & ~total['price_value'].isna()]
train_30 = total[total['car_id'].isin(test_car_ids)]

print(len(total.index))
print(len(total[~total['price_value'].isna()].index))

print(train_70['price_value'].isna().sum())
print(train_30['price_value'].isna().sum())

train_100 = total[~total['price_value'].isna()]
print(train_100['price_value'].isna().sum())

global_min = total[~total['price_value'].isna()].price_value.min()

test = total[total['price_value'].isna()]
train = total[~total['price_value'].isna()]


/home/conda/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,86,161) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160616 entries, 0 to 160615
Data columns (total 218 columns):
 #   Column                                                  Dtype  
---  ------                                                  -----  
 0   breadcrumb                                              object 
 1   created_at                                              object 
 2   price_currency                                          object 
 3   price_details                                           object 
 4   price_value                                             float64
 5   seller_address                                          object 
 6   seller_name                                             object 
 7   seller_type                                             object 
 8   feature_czujniki_parkowania_przednie                    bool   
 9   feature_poduszka_powietrzna_chroniąca_kolana            bool   
 10  feature_kurtyny_powietrzne                             

160616
106494
0
0
0


In [9]:
start_vars = np.sort([
# 'created_at_unix', 'encoded_param_napęd_2', 'encoded_param_rodzaj_paliwa_2', 'encoded_param_skrzynia_biegów_2', 'feature_abs', 'feature_alufelgi', 'feature_asr__kontrola_trakcji_', 'feature_asystent_parkowania', 'feature_bluetooth', 'feature_cd', 'feature_czujniki_parkowania_przednie', 'feature_elektryczne_szyby_tylne', 'feature_elektrycznie_ustawiane_lusterka', 'feature_esp__stabilizacja_toru_jazdy_', 'feature_gniazdo_sd', 'feature_klimatyzacja_automatyczna', 'feature_klimatyzacja_dwustrefowa', 'feature_klimatyzacja_manualna', 'feature_radio_fabryczne', 'feature_tempomat', 'feature_światła_do_jazdy_dziennej', 'feature_światła_led', 'feature_światła_xenonowe', 'latitude', 'longitude', 'model_XGBRegressor_5856_36229_', 'new_param_kod_silnika_cat', 'new_param_liczba_pozostałych_rat_cat', 'new_param_marka_pojazdu_cat', 'new_param_model_pojazdu_cat', 'new_param_napęd_cat', 'new_param_oferta_od_cat', 'new_param_vin_cat', 'new_param_wersja_cat', 'new_seller_name_cat', 'param_bezwypadkowy', 'param_ile_lat_temu_produkcja', 'param_kierownica_po_prawej__anglik_', 'param_leasing', 'param_liczba_miejsc_2', 'param_moc', 'param_pierwsza_rejestracja_date_na', 'param_pojemność_skokowa', 'param_przebieg', 'param_rok_produkcji', 'param_serwisowany_w_aso', 'param_tuning', 'param_uszkodzony', 'param_wartość_wykupu_2', 'param_zarejestrowany_w_polsce', 'price_currency_pln'
# 'created_at_unix', 'encoded_param_rodzaj_paliwa_2', 'encoded_param_typ_2', 'feature_abs', 'feature_asr__kontrola_trakcji_', 'feature_bluetooth', 'feature_cd', 'feature_czujniki_parkowania_przednie', 'feature_elektrycznie_ustawiane_fotele', 'feature_elektrycznie_ustawiane_lusterka', 'feature_gniazdo_sd', 'feature_hud__wyświetlacz_przezierny_', 'feature_isofix', 'feature_klimatyzacja_automatyczna', 'feature_klimatyzacja_manualna', 'feature_ogrzewanie_postojowe', 'feature_podgrzewane_tylne_siedzenia', 'feature_relingi_dachowe', 'feature_tapicerka_skórzana', 'feature_tapicerka_welurowa', 'feature_tempomat', 'feature_światła_led', 'feature_światła_przeciwmgielne', 'feature_światła_xenonowe', 'latitude_na_mean', 'longitude', 'longitude_na_mean', 'model_8790', 'new_param_kolor_cat', 'new_param_liczba_pozostałych_rat_cat', 'new_param_marka_pojazdu_cat', 'new_param_oferta_od_cat', 'new_param_opłata_początkowa_cat', 'new_param_rodzaj_paliwa_cat', 'new_param_wersja_cat', 'new_seller_name_cat', 'new_vin_flex_fuel_cat', 'new_vin_model_cat', 'new_vin_security_cat', 'param_bezwypadkowy', 'param_homologacja_ciężarowa', 'param_ile_lat_temu_produkcja', 'param_leasing', 'param_liczba_miejsc_2', 'param_liczba_pozostałych_rat_2', 'param_matowy', 'param_przebieg', 'param_rok_produkcji', 'param_tuning', 'param_uszkodzony', 'param_wartość_wykupu_2'
# 'model_8790', 'feature_bluetooth', 'created_at_unix', 'encoded_param_rodzaj_paliwa_2', 'encoded_param_typ_2', 'feature_asr__kontrola_trakcji_', 'feature_asystent_parkowania', 'feature_cd', 'feature_centralny_zamek', 'feature_elektrycznie_ustawiane_fotele', 'feature_elektrycznie_ustawiane_lusterka', 'feature_gniazdo_sd', 'feature_isofix', 'feature_klimatyzacja_automatyczna', 'feature_klimatyzacja_manualna', 'feature_ogrzewanie_postojowe', 'feature_podgrzewane_tylne_siedzenia', 'feature_relingi_dachowe', 'feature_tapicerka_skórzana', 'feature_tempomat', 'feature_światła_led', 'feature_światła_przeciwmgielne', 'longitude', 'latitude', 'new_param_kolor_cat', 'new_param_liczba_pozostałych_rat_cat', 'new_param_marka_pojazdu_cat', 'new_param_oferta_od_cat', 'new_param_opłata_początkowa_cat', 'new_param_wersja_cat', 'new_seller_name_cat', 'new_vin_flex_fuel_cat', 'new_vin_model_cat', 'new_vin_portrait_cat', 'new_vin_security_cat', 'ogloszenie_rok', 'param_bezwypadkowy', 'param_homologacja_ciężarowa', 'param_ile_lat_temu_produkcja', 'param_leasing', 'param_liczba_miejsc_2', 'param_liczba_pozostałych_rat_2', 'param_matowy', 'param_moc', 'param_perłowy', 'param_pierwsza_rejestracja_10_lat', 'param_przebieg', 'param_rok_produkcji', 'param_tuning', 'param_uszkodzony', 'param_vin_len'
# 'encoded_param_rodzaj_paliwa_2', 'feature_abs', 'feature_asr__kontrola_trakcji_', 'feature_bluetooth', 'feature_centralny_zamek', 'feature_elektrycznie_ustawiane_fotele', 'feature_elektrycznie_ustawiane_lusterka', 'feature_gniazdo_sd', 'feature_immobilizer', 'feature_isofix', 'feature_klimatyzacja_automatyczna', 'feature_klimatyzacja_manualna', 'feature_komputer_pokładowy', 'feature_ogrzewanie_postojowe', 'feature_podgrzewane_tylne_siedzenia', 'feature_poduszka_powietrzna_kierowcy', 'feature_radio_fabryczne', 'feature_szyberdach', 'feature_tapicerka_skórzana', 'feature_tempomat', 'feature_zmieniarka_cd', 'feature_światła_led', 'feature_światła_przeciwmgielne', 'model_8790', 'new_param_kolor_cat', 'new_param_liczba_pozostałych_rat_cat', 'new_param_marka_pojazdu_cat', 'new_param_model_pojazdu_cat', 'new_param_oferta_od_cat', 'new_param_opłata_początkowa_cat', 'new_param_wersja_cat', 'new_seller_name_cat', 'new_vin_flex_fuel_cat', 'new_vin_model_cat', 'new_vin_portrait_cat', 'new_vin_security_cat', 'ogloszenie_rok', 'param_bezwypadkowy', 'param_filtr_cząstek_stałych', 'param_homologacja_ciężarowa', 'param_ile_lat_temu_produkcja', 'param_liczba_pozostałych_rat_2', 'param_matowy', 'param_metalik', 'param_moc', 'param_perłowy', 'param_pierwsza_rejestracja_10_lat', 'param_pierwsza_rejestracja_ten_rok', 'param_przebieg', 'param_serwisowany_w_aso', 'param_tuning'
# 'encoded_param_rodzaj_paliwa_2', 'feature_abs', 'feature_bluetooth', 'feature_centralny_zamek', 'feature_elektrycznie_ustawiane_fotele', 'feature_elektrycznie_ustawiane_lusterka', 'feature_gniazdo_sd', 'feature_immobilizer', 'feature_isofix', 'feature_klimatyzacja_automatyczna', 'feature_klimatyzacja_manualna', 'feature_komputer_pokładowy', 'feature_ogrzewanie_postojowe', 'feature_podgrzewane_tylne_siedzenia', 'feature_poduszka_powietrzna_kierowcy', 'feature_radio_fabryczne', 'feature_szyberdach', 'feature_tapicerka_skórzana', 'feature_tempomat', 'feature_wielofunkcyjna_kierownica', 'feature_zmieniarka_cd', 'feature_światła_led', 'feature_światła_przeciwmgielne', 'model_8790', 'new_param_kategoria_cat', 'new_param_kolor_cat', 'new_param_liczba_pozostałych_rat_cat', 'new_param_marka_pojazdu_cat', 'new_param_model_pojazdu_cat', 'new_param_oferta_od_cat', 'new_param_opłata_początkowa_cat', 'new_seller_name_cat', 'new_vin_flex_fuel_cat', 'new_vin_model_cat', 'new_vin_portrait_cat', 'new_vin_security_cat', 'ogloszenie_rok', 'param_faktura_vat', 'param_filtr_cząstek_stałych', 'param_homologacja_ciężarowa', 'param_ile_lat_temu_produkcja', 'param_liczba_pozostałych_rat_2', 'param_matowy', 'param_metalik', 'param_moc', 'param_perłowy', 'param_pierwsza_rejestracja_10_lat', 'param_pierwsza_rejestracja_ten_rok', 'param_przebieg', 'param_serwisowany_w_aso', 'param_tuning'
'encoded_param_napęd_2', 'encoded_param_rodzaj_paliwa_2', 'feature_asystent_parkowania', 'feature_centralny_zamek', 'feature_czujnik_deszczu', 'feature_elektrochromatyczne_lusterko_wsteczne', 'feature_elektryczne_szyby_tylne', 'feature_elektrycznie_ustawiane_fotele', 'feature_elektrycznie_ustawiane_lusterka', 'feature_hud__wyświetlacz_przezierny_', 'feature_klimatyzacja_manualna', 'feature_mp3', 'feature_nawigacja_gps', 'feature_odtwarzacz_dvd', 'feature_ogranicznik_prędkości', 'feature_ogrzewanie_postojowe', 'feature_podgrzewana_przednia_szyba', 'feature_podgrzewane_tylne_siedzenia', 'feature_poduszka_powietrzna_kierowcy', 'feature_poduszki_boczne_przednie', 'feature_poduszki_boczne_tylne', 'feature_przyciemniane_szyby', 'feature_radio_fabryczne', 'feature_regulowane_zawieszenie', 'feature_tapicerka_skórzana', 'feature_wielofunkcyjna_kierownica', 'feature_światła_led', 'feature_światła_przeciwmgielne', 'feature_światła_xenonowe', 'model_8790', 'new_param_emisja_co2_cat', 'new_param_kategoria_cat', 'new_param_rodzaj_paliwa_cat', 'new_param_skrzynia_biegów_cat', 'new_param_stan_cat', 'new_param_wersja_cat', 'new_seller_type_cat', 'new_seller_wojewodzctwo_cat', 'new_vin_portrait_cat', 'new_vin_security_cat', 'param_bezwypadkowy', 'param_faktura_vat', 'param_filtr_cząstek_stałych', 'param_liczba_miejsc_2', 'param_liczba_pozostałych_rat_2', 'param_miesięczna_rata_2', 'param_pierwsza_rejestracja_10_lat', 'param_pierwsza_rejestracja_date_na', 'param_przebieg', 'param_serwisowany_w_aso', 'param_vin_len'
])

# 'model_8790' = car_price_log_Vladimir_categories_xgb.csv; 5637.56722; "opis": "Vladimir z kategorialnymi mae na train_30"

black_list = ['car_id', 'price_details', 'price_value', 'price_value_log', 'price_value_pln', 'price_value_pln_log_transl', 'new_cat_price_value',
#               *[x for x in total.columns if x.startswith('model_')],
              'model_XGBRegressor_2195_24982_po_hiperopcie_caly_train', #train_100, przeuczony
              'model_XGBRegressor_3293_22097_', #train_100
              'longitude_na_mean', 'latitude_na_mean', #gorsze w xbfir
              *total.columns[total.isnull().any()],
             ]

feats3 = total.select_dtypes(include=[np.number, np.bool]).columns
feats3 = [x for x in feats3 if x not in black_list]
feats3 = np.sort(feats3)

print(total[start_vars].info(verbose=True))

print(total[feats3].info(verbose=True))
print(total[feats3].columns[total[feats3].isnull().any()])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160616 entries, 0 to 160615
Data columns (total 51 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   encoded_param_napęd_2                          160616 non-null  int64  
 1   encoded_param_rodzaj_paliwa_2                  160616 non-null  int64  
 2   feature_asystent_parkowania                    160616 non-null  bool   
 3   feature_centralny_zamek                        160616 non-null  bool   
 4   feature_czujnik_deszczu                        160616 non-null  bool   
 5   feature_elektrochromatyczne_lusterko_wsteczne  160616 non-null  bool   
 6   feature_elektryczne_szyby_tylne                160616 non-null  bool   
 7   feature_elektrycznie_ustawiane_fotele          160616 non-null  bool   
 8   feature_elektrycznie_ustawiane_lusterka        160616 non-null  bool   
 9   feature_hud__wyświetlacz_przezierny_ 

Index([], dtype='object')


In [12]:
'model_8790' in current_vars

True

In [15]:
import contextlib
import warnings
warnings.filterwarnings("ignore")
import json

params = {    
    'n_jobs': 12, 

    'n_estimators': 90, 
    'learning_rate': 0.1, 
    'subsample': 0.5, 
    'colsample_bytree': 0.9,
    'max_depth': 10, 
}

mae_global_min = 2539.081044523367
DELTA = 0.0

big_iter = 0

current_vars=start_vars
all_vars = feats3 #maksymalny set

for variable in all_vars:

    if variable not in current_vars:
        print('Testuje: ' + variable)
        current_vars = np.sort([*current_vars, variable]) 
        print('len: ' + str(len(current_vars)))
        
        mae, r2, model, model_cl, learning_curve_plot = mozo2.train_and_submit(train_70, train_30, total.loc[total['price_value'].isna(),], 
                     current_vars, global_min, 
                     xgb.XGBRegressor, params, digitize=None, subfolder='XGBRegressor_forward',
                     file_sufix = 'backward_1_zm', opis='start=2604.46495618895, delta', 
                     learning_curve = False, 
                     add_model_column_min = -1, total = total, 
                     save_min=mae_global_min, kaggle_min=-1)

        if mae < mae_global_min + DELTA:
            if (mae < mae_global_min):
                mae_global_min = mae
                print('Improvement: ' + variable)
            else:
                print('Mutation: ' + variable)
        
        current_vars = start_vars
        
        print('=============================================' + str(mae_global_min)+ '=============================================')

print('END')

Testuje: created_at_unix
len: 52
mea: 2541.815397744043
r2: 0.9793261061309825
=============================================2539.081044523367=============================================
Testuje: created_at_unix_na_mean
len: 52
mea: 2541.370432050809
r2: 0.9792588078435528
=============================================2539.081044523367=============================================
Testuje: encoded_param_kolor_2
len: 52
mea: 2533.961327016666
r2: 0.9797801499349643
Improvement: encoded_param_kolor_2
=============================================2533.961327016666=============================================
Testuje: encoded_param_skrzynia_biegów_2
len: 52
mea: 2526.659816396989
r2: 0.9796029653749206
Improvement: encoded_param_skrzynia_biegów_2
=============================================2526.659816396989=============================================
Testuje: encoded_param_typ_2
len: 52
mea: 2522.5684193122147
r2: 0.9799527497117682
Improvement: encoded_param_typ_2
=========================

mea: 2534.2512720917885
r2: 0.9797198351443283
=============================================2510.5911186788753=============================================
Testuje: feature_tempomat
len: 52
mea: 2528.1739092198204
r2: 0.979839051397346
=============================================2510.5911186788753=============================================
Testuje: feature_tempomat_aktywny
len: 52
mea: 2530.245565601822
r2: 0.9796917387199395
=============================================2510.5911186788753=============================================
Testuje: feature_tuner_tv
len: 52
mea: 2525.7914568996466
r2: 0.9799558731904731
=============================================2510.5911186788753=============================================
Testuje: feature_wspomaganie_kierownicy
len: 52
mea: 2531.655206899035
r2: 0.9797279728083341
=============================================2510.5911186788753=============================================
Testuje: feature_zmieniarka_cd
len: 52
mea: 2525.0033796643706
r2

mea: 2642.5941930494787
r2: 0.979371763488116
=============================================2510.5911186788753=============================================
Testuje: param_ile_lat_temu_produkcja
len: 52
mea: 2584.4377406338303
r2: 0.9796220704367742
=============================================2510.5911186788753=============================================
Testuje: param_kierownica_po_prawej__anglik_
len: 52
mea: 2648.425868456628
r2: 0.9791520276746422
=============================================2510.5911186788753=============================================
Testuje: param_leasing
len: 52
mea: 2639.7625054663995
r2: 0.9794773106945747
=============================================2510.5911186788753=============================================
Testuje: param_liczba_drzwi
len: 52
mea: 2656.7823039477958
r2: 0.979208808836679
=============================================2510.5911186788753=============================================
Testuje: param_matowy
len: 52
mea: 2639.0567740437364
r2:

In [16]:
from kaggle.api.kaggle_api_extended import KaggleApi
kaggle_api = KaggleApi()
kaggle_api.authenticate()

leaderboard = kaggle_api.competition_view_leaderboard('dw-car-price-prediction')
print(pd.DataFrame(leaderboard['submissions'])[['teamName', 'submissionDate', 'score']].head())

          teamName                submissionDate       score
0       adamskorek  2020-11-01T15:37:13.6066667Z  5071.64645
1      Paweł Grześ       2020-11-01T11:20:39.31Z  5337.95298
2           pelele  2020-10-31T19:46:31.2633333Z  5369.00636
3             mozo  2020-10-30T16:38:46.7333333Z  5466.72056
4  Adrian Mich***i  2020-11-01T13:58:39.9366667Z  5571.34801


In [38]:
# model = RandomForestRegressor(n_estimators=50, max_depth=7).fit(X_sub_train, y_sub_train) 
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.svm import SVC


# perm = PermutationImportance(model).fit(X_sub_test, y_sub_test) 
eli5.show_weights(perm, feature_names=random_feats, top=50) 


/home/conda/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/conda/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


NameError: name 'perm' is not defined

save_model(file_sufix='test', subfolder='', test = total.loc[total['price_value'].isna()],
               feats=current_vars, global_min=global_min, model=model, shift = 0, digitize=False, mae=mae, r2=r2,
               details='test zapisu', learning_curve_plot=None, hist_plt=None, kaggle=None)
